In [5]:
import json
import requests

url = 'https://api.github.com/some/endpoint'
payload = {'some': 'data'}

r = requests.post(url, data=json.dumps(payload))
r

<Response [404]>

**Q: [Difference between data and json parameters in python requests package](https://stackoverflow.com/questions/26685248/difference-between-data-and-json-parameters-in-python-requests-package)**

**A:** it appears my two examples above do the same thing and that using the json parameter does indeed set the content-type in the headers to application/json. In my first example above using the data parameter, the content-type in the headers would need to be set manually.

In [ ]:
print(f"url.headers = {json.dumps(dict(response.headers), indent=4)}")

Q: 给定时间和limit，实现rate limiting. 比如，同一个user，2秒内只能call 5次

In [20]:
# Q1: do we care about the accuracy? This implementation doesn't care accuracy.
# Q2: is declined call counting towards the limit of 5 later? This implementation doesn't count this.

from collections import defaultdict
N = 2

class RateLimiter():
    def __init__(self, N, limit):
        self.N = N
        self.limit = limit
        self.times = [0] * N
        self.buckets = [defaultdict(int)] * N
    
    def accept(self, user_name, timestamp):
        idx = timestamp % self.N
        if timestamp != self.times[idx]:
            self.buckets[idx] = defaultdict(int)
            self.times[idx] = timestamp
        total = 0
        for i in range(self.N):
            if timestamp - self.times[i] < self.N:
                total += self.buckets[i].get(user_name, 0)
            else:
                self.buckets[i] = defaultdict(int)
        if total >= 5:
            return False
        self.buckets[idx][user_name] += 1
        return True


rate_limiter = RateLimiter(2, 5)
print(rate_limiter.accept("a", 1))
print(rate_limiter.accept("a", 1))
print(rate_limiter.accept("a", 2))
print(rate_limiter.accept("a", 2))
print(rate_limiter.accept("a", 2))
print(rate_limiter.accept("a", 2))
print(rate_limiter.accept("a", 3))
print(rate_limiter.accept("a", 3))
print(rate_limiter.accept("a", 3))




True
True
True
True
True
False
True
True
False


In [21]:
from collections import defaultdict
d = defaultdict(int)
d["a"] += 1
d

defaultdict(int, {'a': 1})

Q: Coding: 这一轮貌似是新题。    完成一个简单的invoice email reminder功能. 实现Invoicer Class, 构造函数给定一个发邮件的schedule {time_diff: email_template}.
    - 实现一个函数send_emails， 给定list of invoice, 每个invoice包含invoice_time, user, due_amount 信息。return 按时间顺序排列的需要发送给user的所有email reminder
    - followup是升级版的send_emails，除了list of invoice以外，还提供一个payment的list。每个payment有一个pay_time, user, pay_amount。return 按时间顺序排列的需要发送给user的所有email reminder。主要区别是已经付清了的invoice就不需要发邮件了。我这题的两问，都用priorityqueue解决的。


Q：coding轮：
有一对银行账号，以及每个账号对应的余额。要求每个账号最后余额不能低于100。问你要进行哪几个transaction，能到达要求。一开始先不要求任何optimization，只要能满足条件就行。
比如{AC1: 120, AC2: 70: AC3: 150, AC4: 80}

那么其中一个合格的输出：["AC1 send 20 to AC2", "AC3 send 10 to AC2", "AC3 send 20 to AC4", ]

In [27]:
from collections import deque

def transaction(accounts):
    if sum([a[1] for a in accounts]) / len(accounts) < 100:
        return "-1"
    rich, poor, res = deque(), deque(), []
    # divide rich and poor
    for acc in accounts:
        acc[1] -= 100
        if acc[1] > 0:
            rich.append(acc)
        elif acc[1] < 0:
            poor.append(acc)
    # give poor money
    while poor:
        if rich[0][1] == abs(poor[0][1]): # rich队列头部账户余额刚好可以抵消poor队列头部账户差值
            res.append(f"{rich[0][0]} send {rich[0][1]} to {poor[0][0]}")
            rich.popleft()
            poor.popleft()
        elif rich[0][1] > abs(poor[0][1]): # rich队列头部账户余额比poor队列头部账户差值多
            res.append(f"{rich[0][0]} send {-poor[0][1]} to {poor[0][0]}")
            rich[0][1] -= abs(poor[0][1])
            poor.popleft()
        else: # rich队列头部账户余额不够支付poor队列头部账户差值
            res.append(f"{rich[0][0]} send {rich[0][1]} to {poor[0][0]}")
            poor[0][1] += rich[0][1]
            rich.popleft()
    return res

print(transaction([['AC1', 120], ['AC2', 70], ['AC3', 150], ['AC4', 80]]))

['AC1 send 20 to AC2', 'AC3 send 10 to AC2', 'AC3 send 20 to AC4']


In [53]:
from collections import deque

def transaction(accounts):
    if sum([a[1] for a in accounts]) / len(accounts) < 100:
        return "-1"
    rich, poor, res = deque(), deque(), []
    # divide rich and poor
    for acc in accounts:
        acc[1] -= 100
        if acc[1] > 0:
            rich.append(acc)
        elif acc[1] < 0:
            poor.append(acc)
    # give poor money
    while poor:
        if rich[0][1] == abs(poor[0][1]): # rich队列头部账户余额刚好可以抵消poor队列头部账户差值
            res.append(f"{rich[0][0]} send {rich[0][1]} to {poor[0][0]}")
            rich.popleft()
            poor.popleft()
        elif rich[0][1] > abs(poor[0][1]): # rich队列头部账户余额比poor队列头部账户差值多
            res.append(f"{rich[0][0]} send {-poor[0][1]} to {poor[0][0]}")
            rich[0][1] -= abs(poor[0][1])
            poor.popleft()
        else: # rich队列头部账户余额不够支付poor队列头部账户差值
            res.append(f"{rich[0][0]} send {rich[0][1]} to {poor[0][0]}")
            poor[0][1] += rich[0][1]
            rich.popleft()
    return res

print(transaction([['AC1', 120], ['AC2', 70], ['AC3', 150], ['AC4', 80]]))

['AC1 send 20 to AC2', 'AC3 send 10 to AC2', 'AC3 send 20 to AC4']


In [39]:
from collections import Counter
a = [1,2,3,4,1,1,2,3,4,7,9,2]
count = Counter(a)
print(count)
print(list(count.items()))

Counter({1: 3, 2: 3, 3: 2, 4: 2, 7: 1, 9: 1})
[(1, 3), (2, 3), (3, 2), (4, 2), (7, 1), (9, 1)]


In [52]:
import heapq 
a = [1,3,54,1,3]
heapq.heapify(a)
print(a)
print(heapq.heappop(a))
print(a)

[1, 1, 54, 3, 3]
1
[1, 3, 54, 3]


Element(10),Element(11),Element(12),Element(13),Element(14),Element(15),Element(16),Element(17),Element(18),Element(19)
Element(19),Element(18),Element(17),Element(16),Element(15),Element(14),Element(13),Element(12),Element(11),Element(10)
